Tuning of PLM across different $\mu$ and $\sigma$ values

In [ ]:
import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config
from phdconf.config import *

%matplotlib inline

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)
law, fact, generic = load_query_focus_types(queries)

In [ ]:
index_names = ['filtered-phrasestop']#, config.SIGIR_INDEX_NAME]
qrel_paths = [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['aus']#, 'SIGIR']

In [ ]:
mu = 1050
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [ ]:
start = 10
end = 200
inc = 10

# plm_dfs = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'plm'), 'case-topics-{0}-plm-dir-mu-300.00-sigma-{1:.2f}.run', rel_levels[0], start, end, inc)

def load_plm(ind, qrel_path, str_format, results_path, rel_level, per_query=False):
    dfs = []
    # sigma
    for s in range(10, 510, 10):
        temp = []
        for mu in range(300, 2050, 50):
            temp.append((s, mu, to_trec_df(qrel_path, 
              os.path.join(results_path, str_format.format(ind, mu, s)), rel_level, per_query=per_query)))        
        dfs.append(temp)
    
    return dfs 

plm_dfs = load_plm(index_names[0], qrel_paths[0], 'case-topics-{0}-plm-dir-mu-{1:.2f}-sigma-{2:.2f}.run', os.path.join(BASE_DIR, 'plm'), '1')

In [ ]:
# metrics = config.METRIC_NAMES
# # del metrics['recall_100']
# # plot = plot_tune_1d_comp(['baseline', 'plm'], metrics, [[base_df[0][0] for x in range(len(plm_dfs[0]))], plm_dfs[0]], start, end, inc)
# plot = plot_tune_1d_comp(display_names, config.METRIC_NAMES, plm_dfs, start, end, inc)

def plot_plm_tuning(dfs, metric_names):
    if len(metric_names) == 1:
        r = 1
        c = 1
    elif len(metric_names) == 2:
        r = 1
        c = 2
    else:
        r = int(len(metric_names)/2)
        c = r
        if c == r and r > 2: 
            r-=1
        if len(metric_names)%2 != 0:
            c += 1 
        
    fig, axs = plt.subplots(r, c, subplot_kw=dict(projection='3d'))
    fig.set_size_inches(12, 8)
    
    num_x = len(dfs[0])
    num_y = len(dfs)
    
    x = np.array([[i]*num_x for i in np.arange(10, 510, 10)])
    y = np.array([list(np.arange(300, 2050, 50))] * num_y)
    
    cnt = 0 
    for i, m in enumerate(metric_names): 
        z = [[y[2][m] for y in x] for x in dfs]
        z = np.array(z)

        axs.plot_surface(x, y, z, cmap=cm.gray)

        axs.tick_params(labelsize=20)
        axs.zaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        axs.invert_xaxis()
        axs.view_init(30,50)
        axs.tick_params(axis='z', pad=10)
        axs.set_zlabel(metric_names[m], fontsize=30, rotation=91, labelpad=20)
        axs.set_ylabel('$\mu$', fontsize=30, labelpad=15)
        axs.set_xlabel('$\sigma$', fontsize=30, labelpad=15)
        cnt += 1

    # if len(metric_names) % 2 != 0: 
    #     fig.delaxes(axs[row, -1])
            
#     fig.subplots_adjust(wspace=0.07, hspace=0.05, left=0.01, right=1.7)
    fig.tight_layout()
    return fig 

plm_fig = plot_plm_tuning(plm_dfs, {'rbp@0.80': 'RBP'})

In [ ]:
# plm_fig.savefig('figures/plm-tune.pdf')

In [ ]:
def read_folds(path: str):
    folds = []
    with open(path) as f:
        for line in f:
            parts = line.split('] [') 
            test = list(map(int, parts[1].replace(']', '').split(', ')))
            train = list(map(int, parts[0].replace('[', '').split(', ')))
            folds.append((test, train))
            
    return folds
            
tt_folds = read_folds(os.path.join('../eval-folds', 'ausnl-folds.txt'))

In [ ]:
plm_qry_dfs = load_plm(index_names[0], qrel_paths[0], 'case-topics-{0}-plm-dir-mu-{1:.2f}-sigma-{2:.2f}.run', os.path.join(BASE_DIR, 'plm'), '1', per_query=True)

In [ ]:
flattened = [x[2] for y in plm_qry_dfs for x in y]

In [ ]:
cv_df = pd.DataFrame(columns=config.METRIC_NAMES)

for ab, runs in zip(['PLM'], [flattened]):
    cross = cross_validation(runs, tt_folds, config.METRIC_NAMES, base_qry)
    cv_df.loc[ab] = cross[0]
    
cv_df.loc['R'] = ['{:.4f}'.format(base_df[m]) for m in config.METRIC_NAMES]
print(bold_max(cv_df).drop('unjudged@20',axis='columns').rename(config.METRIC_NAMES, axis='columns').to_latex(escape=False))

In [ ]:
def create_max_df(dfs): 
#     mu_start = 300
#     mu_end = 2050
#     mu_inc = 50
    
#     sig_start = 10
#     sig_end = 500
#     sig_inc = 10
    
#     num_x = len(dfs[0])
#     num_y = len(dfs)
    
#     x = np.array([[i]*num_x for i in np.arange(sig_start, sig_end, sig_inc)])
#     y = np.array([list(np.arange(mu_start, mu_end, mu_inc))] * num_y)
    
    max_for = {}
    # for diff values of sig 
    for i in range(len(dfs)):
        for j in range(len(dfs[i])):
            inds = dfs[i][j][2].index
            for ind in inds:
                if ind not in max_for:
                    max_for[ind] = (dfs[i][j][2][ind], dfs[i][j][0], dfs[i][j][1])
                else:
                    if dfs[i][j][2][ind] > max_for[ind][0]:
                        max_for[ind] = (dfs[i][j][2][ind], dfs[i][j][0], dfs[i][j][1])

    for measure in max_for:
        max_for[measure] = {'\mu': max_for[measure][2], '\sigma': max_for[measure][1], 'score': max_for[measure][0]}
        
    max_df = pd.DataFrame.from_dict(max_for, orient='index')
    return max_df

max_df = create_max_df(plm_dfs)

In [ ]:
print(max_df[max_df.index.isin(config.METRIC_NAMES)].rename(index=config.METRIC_NAMES).round(4).to_latex())

In [ ]:
# SIGIR SDM

sig_plm = load_dfs(config.SIGIR_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'plm', 'sigir-topic-topics-sigir-stop-plm-dir-mu-1500.00-sigma-{0:.2f}.run'.format(x)) for x in range(25, 525, 25)], filtered=[1, 3, 9, 19, 23, 25, 31, 37, 39, 45, 51, 97])

In [ ]:
sig_plot = plot_tune_1d_comp(['plm'], config.METRIC_NAMES, [sig_plm], 25, 500, 25)
sig_plot.savefig('figures/sigir-plm-tune.pdf')

In [ ]:
sig_plm = load_dfs(config.SIGIR_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'plm', 'sigir-topic-topics-sigir-stop-plm-dir-mu-1500.00-sigma-{0:.2f}.run'.format(x)) for x in range(25, 525, 25)], per_query=True, filtered=[1, 3, 9, 19, 23, 25, 31, 37, 39, 45, 51, 97])

In [ ]:
base_sig = load_1d_dfs(['sigir-stop'], [config.SIGIR_QREL_PATH], os.path.join(BASE_DIR, 'dirichlet_prior'), 'sigir-topic-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1500, 1500, 1, per_query=True, filtered=[1, 3, 9, 19, 23, 25, 31, 37, 39, 45, 51, 97])[0][0]

sig_folds = read_folds('sigir-folds.txt')

In [ ]:
cv_df = pd.DataFrame(columns=config.METRIC_NAMES)


def cross_validation(runs, folds, metrics, base_qry):
    _max = [0.0] * len(metrics)
    max_inds = [0] * len(metrics)
    qry_res = []
    for m in metrics:
        qry_res.append(pd.Series(dtype='float64'))
    
    for i, run in enumerate(runs):
        for fold in folds: 
            filtered = run[run.index.isin(fold[1])]
            for j, m in enumerate(metrics.keys()):
                v = filtered[m].mean()
                # if j == 0: 
                #     print(v)
                if v > _max[j] or i == 0:
                    _max[j] = v
                    max_inds[j] = i
                    for ind, item in run[run.index.isin(fold[0])][m].items():
                        qry_res[j].loc[ind] = item
    
    for i, m in enumerate(metrics):
        qry_res[i].sort_index(inplace=True)
        p = stats.ttest_rel(qry_res[i], base_qry[m]).pvalue
        mean = base_qry[m].mean()
        if p < 0.01:
            _max[i] = '{0:.4f}'.format((_max[i])) + "$^{**}$"
        elif p < 0.05: 
            _max[i] = '{0:.4f}'.format((_max[i])) + "$^{*}$"
        else:
            _max[i] = '{0:.4f}'.format((_max[i]))

    return _max, max_inds


for ab, runs in zip(['PLM'], [sig_plm]):
    cross = cross_validation(runs, sig_folds, config.METRIC_NAMES, base_sig)
    cv_df.loc[ab] = cross[0]
    
cv_df.loc['R'] = ['{:.4f}'.format(base_sig[m].mean()) for m in config.METRIC_NAMES]
print(cv_df.drop('unjudged@20',axis='columns').rename(config.METRIC_NAMES, axis='columns'))